In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
279,AFG,Asia,Afghanistan,2020-10-05,39341.0,44.0,16.286,1462.0,0.0,1.286,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83,0.498
490,ALB,Europe,Albania,2020-10-05,14266.0,149.0,143.857,396.0,4.0,2.714,...,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57,0.785
770,DZA,Africa,Algeria,2020-10-05,52136.0,141.0,152.714,1760.0,4.0,6.571,...,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88,0.754


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-10-04,France,618603.0,0.0,32318.0,0.0,0.0,0.0
0,2020-10-04,China,91281.0,0.0,4744.0,0.0,0.0,0.0
0,2020-10-04,Italy,324807.0,0.0,36004.0,0.0,0.0,0.0
0,2020-10-04,Spain,800034.0,0.0,32177.0,0.0,0.0,0.0
0,2020-10-04,United States,7451779.0,0.0,210576.0,0.0,0.0,0.0
0,2020-10-04,World,35495713.0,0.0,1043484.0,0.0,0.0,0.0
0,2020-10-04,United Kingdom,485061.0,0.0,42396.0,0.0,0.0,0.0
0,2020-10-04,Germany,301408.0,0.0,9549.0,0.0,0.0,0.0
0,2020-10-04,Iran,471370.0,0.0,26977.0,0.0,0.0,0.0
0,2020-10-04,Turkey,324167.0,0.0,8437.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.at[0,"total_cases_predict"]
    rez2 = res_tempo.at[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.at[0,"date"] = v0
    follow_df.at[0,"country"] = country
    follow_df.at[0,"total_cases_predict"] = rez1
    follow_df.at[0,"total_cases_real"] = v1
    follow_df.at[0,"total_deaths_predict"] = rez2
    follow_df.at[0,"total_deaths_real"] = v2
    follow_df.at[0,"error_abs_cases"] = errorCase1
    follow_df.at[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-10-05,France,618603.0,619170.0,32318.0,32230.0,-567.0,88.0
0,2020-10-05,China,91281.0,90629.0,4744.0,4739.0,652.0,5.0
0,2020-10-05,Italy,324807.0,325329.0,36004.0,35986.0,-522.0,18.0
0,2020-10-04,Spain,800034.0,789932.0,32177.0,32086.0,10102.0,91.0
0,2020-10-05,United States,7451779.0,7417845.0,210576.0,209794.0,33934.0,782.0
0,2020-10-05,World,35495713.0,35247104.0,1043484.0,1038069.0,248609.0,5415.0
0,2020-10-05,United Kingdom,485061.0,502978.0,42396.0,42350.0,-17917.0,46.0
0,2020-10-05,Germany,301408.0,300619.0,9549.0,9534.0,789.0,15.0
0,2020-10-05,Iran,471370.0,471772.0,26977.0,26957.0,-402.0,20.0
0,2020-10-05,Turkey,324167.0,324443.0,8437.0,8441.0,-276.0,-4.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")